In [3]:
import sncosmo

In [4]:
from analyzeSN import SNANASims

In [5]:
# read in simulation
snana_eg = SNANASims.fromSNANAfileroot(snanafileroot='LSST_Ia',
                                       location='/Users/rbiswas/data/LSST/SNANA_data/MINION_1016_10YR_DDF_v2/',
                                       coerce_inds2int=False)

In [6]:
snana_eg.headData.head()

,IAUC,FAKE,RA,DECL,PIXSIZE,NXPIX,NYPIX,SNTYPE,NOBS,PTROBS_MIN,...,SIM_EXPOSURE_r,SIM_EXPOSURE_i,SIM_EXPOSURE_z,SIM_EXPOSURE_Y,SIM_GALFRAC_u,SIM_GALFRAC_g,SIM_GALFRAC_r,SIM_GALFRAC_i,SIM_GALFRAC_z,SIM_GALFRAC_Y
SNID,,,,,,,,,,,,,,,,,,,,,
13,NULL,2,0.000000,-45.524532,0.2,-9,-9,1,71,1,...,1.0,1.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0
31,NULL,2,150.362350,2.836485,0.2,-9,-9,1,90,73,...,1.0,1.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0
42,NULL,2,34.393394,-5.090329,0.2,-9,-9,1,80,164,...,1.0,1.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0
61,NULL,2,349.386444,-63.321003,0.2,-9,-9,1,68,245,...,1.0,1.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0
89,NULL,2,150.362350,2.836485,0.2,-9,-9,1,76,314,...,1.0,1.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,-9.0,-9.0


In [7]:
snids = snana_eg.headData.index.values

In [8]:
lc = snana_eg.get_SNANA_photometry(snid='31')

In [9]:
max(lc.lightCurve['flux']/lc.lightCurve['fluxerr'] )

4.1012692

In [10]:
from analyzeSN import ResChar

In [11]:
def inferParams(snanaSims, model, infer_method, i, minsnr=3.):
    """
    infer the parameters for the ith supernova in the simulation
    """
    snid = snanaSims.headData.index.values[i]
    z = snanaSims.headData.ix[snid, 'REDSHIFT_FINAL']
    lcinstance = snanaSims.get_SNANA_photometry(snid=snid)
    model.set(z=z)
    print(z)
    resfit = infer_method(lcinstance.snCosmoLC(), model, vparam_names=['t0', 'x0', 'x1', 'c'],
                          modelcov=True, minsnr=minsnr)
    reschar = ResChar.fromSNCosmoRes(resfit)
    return reschar

In [12]:
dust = sncosmo.CCM89Dust()
model = sncosmo.Model(source='salt2-extended', effects=[dust, dust],
                     effect_names=['host', 'mw'], effect_frames=['rest', 'obs'])

In [13]:
#example
r = inferParams(snana_eg, model, sncosmo.fit_lc, 0)

1.16624


In [14]:
r.parameters

z          1.166243e+00
t0         6.096514e+04
x0         4.229721e-07
x1        -4.430033e+00
c         -3.031123e-01
hostebv    0.000000e+00
hostr_v    3.100000e+00
mwebv      0.000000e+00
mwr_v      3.100000e+00
dtype: float64

In [15]:
r.covariance

,t0,x0,x1,c
t0,6.221200e-02,-9.897254e-10,1.119111e-02,-2.054025e-03
x0,-9.897254e-10,7.092694e-15,-7.058787e-11,1.042686e-08
x1,1.119111e-02,-7.058787e-11,9.346116e-03,3.397151e-04
c,-2.054025e-03,1.042686e-08,3.397151e-04,3.238695e-02


In [ ]:
r = inferParams(snana_eg, model, sncosmo.fit_lc, 1)

0.699517


In [1]:
r.parameters

NameError: name 'r' is not defined

In [ ]:
r.covariance

In [ ]:
r = inferParams(snana_eg, model, sncosmo.fit_lc, 0, minsnr=5.)

PSEUDO CODE (SERIAL)

- read in simulation
- create model
- iterate over number of SN:
    - try :
        - call inferParams : r = inferParams(...)
        - write r.params, r.covariance (after packing to 10 or 15 independent numbers), r.samples (if exists, desired)
    - except :
        failures

PSEUDO CODE (Parallel ?)

- read in simulation and create chunks

- Distribute chunks to nodes
- On each node 
    - iterate over number of SN (more SN than nodes :) ):
    - create model
        - try :
            - call inferParams : r = inferParams(...)
            - write r.params, r.covariance (after packing to 10 or 15 independent numbers), r.samples (if exists, desired)
        - except :
            failures

In [ ]:
def inferParams(snanaSims, model, infer_method, i, minsnr=3.):
    """
    infer the parameters for the ith supernova in the simulation
    """
    snid = snanaSims.headData.index.values[i]
    z = snanaSims.headData.ix[snid, 'REDSHIFT_FINAL']
    lcinstance = snanaSims.get_SNANA_photometry(snid=snid)
    model.set(z=z)
    print(z)
    resfit = infer_method(lcinstance.snCosmoLC(), model, vparam_names=['t0', 'x0', 'x1', 'c'],
                          modelcov=True, minsnr=minsnr)
    reschar = ResChar.fromSNCosmoRes(resfit)
    return reschar

snana_eg = SNANASims.fromSNANAfileroot(snanafileroot='LSST_Ia',
                                       location='/Users/rbiswas/data/LSST/SNANA_data/MINION_1016_10YR_DDF_v2/',
                                       coerce_inds2int=False)